HyperTS has built-in rich modeling algorithms for different modes, such as:

- StatsForecastSearchSpace: Prophet、ARIMA、VAR;
- StatsClassificationSearchSpace: TSForest, k-NNs;
- DLForecastSearchSpace: DeepAR、RNN、GPU、LSTM、LSTNet;
- DLClassificationSearchSpace: RNN、GPU、LSTM、LSTNet.

The above algorithms are designed with their own default hyperparameter search spaces. If you want to customize your own search space on this basis, you can specify the customized search space through the parameter ```search_space``` when calling ```make_experiment```.

Suppose now we want to modify the search space of the statistical mode under the forecasting task, namely ```StatsForecastSearchSpace```, you can do the following:

- ~Specify the ```task``` in detail, otherwise it is impossible to judge whether it is a univariate or a multivariate forecast task;~
- ~Specify the ```timestamp``` column;~
- ~If there are covariates, set the parameter ```covariables```;~
- ~**Please strictly abide by the above three steps, otherwise the customization will fail!!!**~
- If you wish to disable an algorithm and do not search, you can set the parameter to ```False```. For example, ```enable_arima=False```;
- If you wish to change the initialization of the search space parameters of an algorithm, you can pass the parameter ```xxx_init_kwargs={xxx:xxx, ...}```;
- If you wish your custom parameters to be searchable, you can use ```Choice```, ```Int``` and ```Real``` in ```hypernets.core.search_space```. Where, ```Choice``` supports discrete values, ```Int``` supports integer continuous values, and ```Real``` supports float continuous values. For details, please refer to [Search Space](https://github.com/DataCanvasIO/Hypernets/blob/master/hypernets/core/search_space.py).

**Note**

- We have updated the custom search space and simplified the parameters.
-  See **Simplify**.

In [1]:
from hypernets.core.search_space import Choice, Int, Real
from hyperts.framework.search_space.macro_search_space import StatsForecastSearchSpace

custom_search_space = StatsForecastSearchSpace(task='univariate-forecast', 
                                               timestamp='TimeStamp',
                                               covariables=['HourSin', 'WeekCos', 'CBWD'],
                                               enable_arima=False,
                                               prophet_init_kwargs={
                                                   'seasonality_mode': 'multiplicative',
                                                   'daily_seasonality': Choice([True, False]),
                                                   'n_changepoints': Int(10, 50, step=10),
                                                   'interval_width': Real(0.1, 0.5, step=0.1)}
                                               )

02-18 15:58:24 W hyperts.macro_search_space.py 214 - Tip: If other parameters exist, set them directly. For example, covariables=['is_holiday'].


In [2]:
from hyperts.datasets import load_network_traffic
from sklearn.model_selection import train_test_split

df = load_network_traffic(univariate=True)
train_data, test_data = train_test_split(df, test_size=168, shuffle=False)

In [3]:
from hyperts import make_experiment

experiment = make_experiment(train_data.copy(), 
                             task='univariate-forecast',
                             timestamp='TimeStamp',
                             covariables=['HourSin', 'WeekCos', 'CBWD'],
                             search_space=custom_search_space) 

### Simplify 

#### 1. StatsForecastSearchSpace

In [1]:
from hypernets.core.search_space import Choice, Int, Real
from hyperts.framework.search_space.macro_search_space import StatsForecastSearchSpace

custom_search_space = StatsForecastSearchSpace(enable_arima=False,
                                               prophet_init_kwargs={
                                                   'seasonality_mode': 'multiplicative',
                                                   'daily_seasonality': Choice([True, False]),
                                                   'n_changepoints': Int(10, 50, step=10),
                                                   'interval_width': Real(0.1, 0.5, step=0.1)}
                                               )

In [2]:
from hyperts.datasets import load_network_traffic
from sklearn.model_selection import train_test_split

df = load_network_traffic(univariate=True)
train_data, test_data = train_test_split(df, test_size=168, shuffle=False)

In [3]:
from hyperts import make_experiment

experiment = make_experiment(train_data.copy(), 
                             task='univariate-forecast',
                             timestamp='TimeStamp',
                             covariables=['HourSin', 'WeekCos', 'CBWD'],
                             search_space=custom_search_space) 

#### 2. DLForecastSearchSpace

In [1]:
from hypernets.core.search_space import Choice, Int, Real
from hyperts.framework.search_space.macro_search_space import DLForecastSearchSpace

custom_search_space = DLForecastSearchSpace(enable_deepar=False, 
                                            enable_hybirdrnn=False, 
                                            enable_lstnet=False, 
                                            enable_nbeats=True)

In [2]:
from hyperts.datasets import load_network_traffic
from sklearn.model_selection import train_test_split

df = load_network_traffic(univariate=True)
train_data, test_data = train_test_split(df, test_size=168, shuffle=False)

In [3]:
from hyperts import make_experiment

experiment = make_experiment(train_data.copy(), 
                             task='univariate-forecast',
                             timestamp='TimeStamp',
                             covariables=['HourSin', 'WeekCos', 'CBWD'],
                             search_space=custom_search_space) 

you alse can...

In [4]:
from hypernets.core.search_space import Choice, Int, Real
from hyperts.framework.search_space.macro_search_space import DLForecastSearchSpace

custom_search_space = DLForecastSearchSpace(
    enable_deepar=False, 
    enable_hybirdrnn=True, 
    enable_lstnet=False, 
    enable_nbeats=True,
    hybirdrnn_init_kwargs = {
        'rnn_units': Choice([8, 16, 64]),
        'rnn_type': Choice(['simple_rnn', 'gru', 'lstm'])
    },
    nebeats_init_kwargs = {
        'nb_blocks_per_stack': Choice([2, 3]),
        'hidden_layer_units': Choice([16, 32, 64]),
    }
)

**Note:**
For the selection of hyperparameters, refer to:
    https://hyperts.readthedocs.io/en/latest/contents/0900_api.html#module-hyperts.framework.estimators